# Test KFP Integration

- create an experiment
- create a run
- check run passes

This Notebook is based on the condition Kubeflow pipelines [example](https://github.com/kubeflow/pipelines/blob/master/samples/core/condition/condition_v2.py).

In [1]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

In [2]:
import kfp
import os

from kfp import dsl
from tenacity import retry, stop_after_attempt, wait_exponential

In [3]:
client = kfp.Client()

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [4]:
EXPERIMENT_NAME = "Flip a coin and output tails/heads pipeline"

In [5]:
@dsl.component()
def flip_coin(force_flip_result: str = "") -> str:
    """Flip a coin and output heads or tails randomly."""
    if force_flip_result:
        return force_flip_result
    import random

    result = "heads" if random.randint(0, 1) == 0 else "tails"
    return result

/opt/conda/lib/python3.11/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [6]:
@dsl.component()
def print_msg(msg: str):
    """Print a message."""
    print(msg)

In [7]:
@dsl.pipeline(name="condition-v2")
def condition_pipeline(text: str = "condition test", force_flip_result: str = ""):
    flip1 = flip_coin(force_flip_result=force_flip_result)
    print_msg(msg=flip1.output)

    with dsl.Condition(flip1.output == "heads"):
        flip2 = flip_coin()
        print_msg(msg=flip2.output)
        print_msg(msg=text)

/tmp/ipykernel_356/2573238994.py:6: DeprecationWarning: dsl.Condition is deprecated. Please use dsl.If instead.
  with dsl.Condition(flip1.output == 'heads'):


In [8]:
run = client.create_run_from_pipeline_func(
    condition_pipeline,
    experiment_name=EXPERIMENT_NAME,
)

In [9]:
client.list_experiments().experiments

[{'created_at': datetime.datetime(2023, 11, 21, 10, 35, tzinfo=tzlocal()),
  'description': None,
  'display_name': 'Flip a coin and output tails/heads pipeline',
  'experiment_id': '721a46c5-c6c9-4d28-af04-00a8503673ac',
  'namespace': 'daniela',
  'storage_state': 'AVAILABLE'}]

In [18]:
client.get_run(run.run_id).state

'SUCCEEDED'

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_run_succeeded(client, run_id):
    """Wait for the run to complete successfully."""
    status = client.get_run(run_id).state
    assert status == "SUCCEEDED", f"KFP run in {status} state."

In [ ]:
# fetch KFP experiment to ensure it exists
client.get_experiment(experiment_name=EXPERIMENT_NAME)

assert_run_succeeded(client, run.run_id)